## NSO Example

This notebook is to demonstrate how to access the NSO dataset stored in OEDI's oedi-data-lake bucket in Amazon AWS S3 storage.

Examples are given to read each of the lidar, masts, and loads data files.

The data are stored in a partitioned parquet format. First, we define a convenience function that will help us locate specific files in the dataset.

In [ ]:
import pandas as pd
import s3fs
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
# pyarrow and fastparquet need to be installed via pip or conda


# Define function for reading the specified dataset
def read_dataset(dataset, year=None, month=None, day=None):
    path = f's3://oedi-data-lake/NSO/{dataset}'
    if year:
        if '*' not in year:
            path += f'/year={year}'
    if month:
        if '*' not in month:
            path += f'/month={month}'
    if day:
        if '*' not in day:
            path += f'/day={day}'

    # Check existence
    if not s3fs.S3FileSystem(anon=True).exists(path):
        raise Exception('No data available for that selection.')

    # Get dataframe
    df = pd.read_parquet(path, storage_options={"anon": True})
    return df

### Read Lidar Data

In [ ]:
# Define date to read data (can be multiple options with '*')
year = '2022'
month = '06'
day = '10'

# Read the lidar dataset at specified date
lidar = read_dataset('lidar', year, month, day)

In [ ]:
# Make lidar example plots

fig = plt.figure()   # doppler vs azimuth
mpl.rcParams['lines.markersize'] = 2
plt.suptitle(f'{year} - {month} - {day}')
ax1 = plt.subplot(1, 1, 1)
ax1.set_ylabel('Doppler velocity (m/s)')
ax1.set_xlabel('Azimuth (deg)')
p = ax1.scatter(lidar.Az, lidar.Doppler_filtered,c=lidar.index, alpha=0.5)
N_TICKS = 9
indexes = [lidar.index[i] for i in np.linspace(0,lidar.shape[0]-1,N_TICKS).astype("int64")] 
cb = plt.colorbar(p, orientation='vertical',
                    ticks= lidar[lidar.Range_gate==0].loc[indexes].index.astype("int64"))  
cb.ax.set_yticklabels([index.strftime('%d %b %Y %H:%M') for index in indexes])
ax1.grid(True)
plt.tight_layout()

fig = plt.figure()  # doppler time series 
plt.suptitle(f'{year} - {month} - {day}')
ax1 = plt.subplot(1, 1, 1)
ax1.set_ylabel('Doppler velocity (m/s)')
ax1.plot(lidar.Doppler,".", label = "unfiltered")
ax1.plot(lidar.Doppler_filtered,".", label = "filtered")
plt.legend()
fig.autofmt_xdate()
ax1.grid(True)
plt.tight_layout()

### Read Masts Data

In [ ]:
# Read the infow and wake masts datasets for all dates (no date specified in the read function)

inflow = read_dataset('inflow_mast_1min')   # 20 Hz resolution: read_dataset('inflow_mast_20Hz')
masts = read_dataset('wake_masts_1min')     # 20 Hz resolution: read_dataset('wake_masts_20Hz')

In [ ]:
# Plot time series for all masts at 7m height

fig = plt.figure(figsize=(15,9))
mpl.rcParams['lines.markersize'] = 1

ax1 = plt.subplot(5, 2, 1)
ax1.set_ylabel('Temperature  ($^\circ$C)')

ax2 = plt.subplot(5, 2, 2, sharex = ax1)
ax2.set_ylabel('RH (%)')

ax3 = plt.subplot(5, 2, 3, sharex = ax1)
ax3.set_ylabel('Stability R_f')
ax3.set_ylim(-0.2,0.2)

ax4 = plt.subplot(5, 2, 4, sharex = ax1)
ax4.set_ylabel('Heat flux (W m$^{-2}$)')

ax5 = plt.subplot(5, 2, 5, sharex = ax1)
ax5.set_ylabel('Wind speed 7m (m s$^{-1}$)')

ax6 = plt.subplot(5, 2, 6, sharex = ax1)
ax6.set_ylabel('Wind direction 7m ($^\circ$)')

ax7 = plt.subplot(5, 2, 9, sharex = ax1)
ax7.set_ylabel('length scale $w$ (m)')

ax8 = plt.subplot(5, 2, 10, sharex = ax1)
ax8.set_ylabel('length scale $U$ (m)')

ax9 = plt.subplot(5, 2, 7, sharex = ax1)
ax9.set_ylabel('TI 7m')

ax10 = plt.subplot(5, 2, 8, sharex = ax1)
ax10.set_ylabel('TKE 7m (m$^{2}$ s$^{-2}$)')

ax1.plot(  inflow.Temp_2m  ,"." , label = "Tem 2m")
try:
    ax1.plot(inflow.Temp_3m,'8',label = 'Temp 3.5m', color="C0")
    ax1.plot(inflow.Temp_7m,'s',label = 'Temp 7m', color="C0")   
except:
    pass
ax2.plot(  inflow.RH  ,"." )
ax3.plot(  inflow.R_f  ,"." )
ax4.plot(  inflow.H_S  ,"." )    
ax5.plot(  inflow.wspd_7m  ,".",label='inflow' )
ax6.plot(  inflow.wdir_7m,"." )
ax7.plot(  inflow.ls_w_7m  ,"." )    
ax8.plot(  inflow.ls_U_7m  ,"." )
ax9.plot(  inflow.TI_U_7m  ,"." )    
ax10.plot(  inflow.TKE_7m  ,"." )   

if len(masts) !=0: 
    ax5.plot(  masts.m1_wspd_7m  ,"." ,label='mast 1')
    ax5.plot(  masts.m2_wspd_7m  ,"." ,label='mast 2')
    ax5.plot(  masts.m3_wspd_7m  ,"." ,label='mast 3')                    
    ax6.plot(  masts.m1_wdir_7m  ,"." )
    ax6.plot(  masts.m2_wdir_7m  ,"." )
    ax6.plot(  masts.m3_wdir_7m  ,"." )  
    ax10.plot(  masts.m1_TKE_7m  ,"." )
    ax10.plot(  masts.m2_TKE_7m  ,"." )
    ax10.plot(  masts.m3_TKE_7m  ,"." )  
    ax9.plot(  masts.m1_TI_U_7m  ,"." )
    ax9.plot(  masts.m2_TI_U_7m  ,"." )
    ax9.plot(  masts.m3_TI_U_7m  ,"." )                  

ax5.legend(loc=3, markerscale=3)
ax1.legend(loc=3, markerscale=3)             
for ax in [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10]:
    ax.grid(True)
    
ax8.set_xlabel('Date/ Time (UTC)')
ax7.set_xlabel('Date/ Time (UTC)')

fig.autofmt_xdate()
plt.tight_layout()

### Read Loads Data

In [ ]:
# Read the loads dataset for all dates (no date specified in the read function)

loads = read_dataset('loads_1min')  # 20 Hz resolution: read_dataset('loads_20Hz')

In [ ]:
# Plot loads time series together with inflow data

inflow_filtered = inflow.loc[loads.index]

fig = plt.figure(figsize=(17,9))   
mpl.rcParams['lines.markersize'] = 1

ax1 = plt.subplot(4, 2, 1)
ax1.set_ylabel('Wind speed 3m (m s$^{-1}$)')    

ax6 = plt.subplot(4, 2, 3, sharex=ax1)
ax6.set_ylabel('Wind direction 3m ($^\circ$)')
plt.yticks([0, 90, 180, 270, 360], ['N', 'E', 'S', "w", "N"]) 

ax9 = plt.subplot(4, 2, 4, sharex=ax1)
ax9.set_ylabel('TI 3m')

ax10 = plt.subplot(4, 2, 2, sharex=ax1)
ax10.set_ylabel('TKE 3m (m$^{2}$ s$^{-2}$)')

ax7 = plt.subplot(4, 2, 7, sharex=ax1)
ax7.set_ylabel('Bending moment SO (kNm)')

ax8 = plt.subplot(4, 2, 8, sharex=ax1)
ax8.set_ylabel('Torque moment DO (kNm)')    

ax4 = plt.subplot(4, 2, 6, sharex=ax1)
ax4.set_ylabel('Displacement (mm)')    

ax3 = plt.subplot(4, 2, 5, sharex=ax1)
ax3.set_ylabel('Tilt ($^\circ$)')

try:
    ax1.plot(inflow_filtered.wspd_3m,".",color='black', label = "inflow")
    ax6.plot(inflow_filtered.wdir_3m,".",color='black')
    ax9.plot(inflow_filtered.TI_U_3m,".",color='black')    
    ax10.plot(inflow_filtered.TKE_3m,".",color='black')  
    ax1.legend(fontsize=7, markerscale= 4, loc='center left', bbox_to_anchor=(1, 0.5))
except:
    pass
    
for column in loads[[col for col in loads.columns if ('SO_Bending' in col)  & ('_m' not in col) & ('_s' not in col)]]:
    ax7.plot(loads[column],".", label = loads[column].name[:2]) 
ax7.legend(fontsize=7, markerscale= 4, loc='center left', bbox_to_anchor=(1, 0.5))
for column in loads[[col for col in loads.columns if ('Torque' in col)  & ('_m' not in col) & ('_s' not in col) & ('_C' not in col)]]:
    ax8.plot(loads[column],".", label = loads[column].name[:2]) 
ax8.legend(fontsize=7, markerscale= 4, loc='center left', bbox_to_anchor=(1, 0.5))
for column in loads[[col for col in loads.columns if ('Disp' in col)  & ('_m' not in col) & ('_s' not in col)& ('_raw' not in col)& ('_level' not in col)]]:
    ax4.plot(loads[column],".", label = loads[column].name.replace('_Disp_', ' ')) 
ax4.legend(fontsize=7, markerscale= 4, loc='center left', bbox_to_anchor=(1, 0.5))
ax3.plot(loads.projected_sun_angle.where((loads.projected_sun_angle<90) & (loads.projected_sun_angle>-90)), ".", color='black', label="nom")
for column in loads[[col for col in loads.columns if ('Tilt' in col)  & ('_m' not in col) & ('_s' not in col) & ('SO' in col)& ('_raw' not in col)]]:
    ax3.plot(loads[column],".", label = loads[column].name[:2]) 
ax3.legend(fontsize=7, markerscale= 4, loc='center left', bbox_to_anchor=(1, 0.5))

for ax in [ax1, ax3, ax4, ax6, ax7, ax8, ax9, ax10]:
    ax.grid(True)
    ax.set_xlim(loads.index[0], loads.index[-1])

ax8.set_xlabel('Date/ Time (UTC)')
ax7.set_xlabel('Date/ Time (UTC)')

fig.autofmt_xdate() 
plt.tight_layout() 